In [2]:
import pandas as pd
import numpy as np
# import random
# import math
from tqdm import tqdm
# from IPython.display import clear_output


In [3]:
def ci_summary(my_list, percent=True, Label='Mean'):
  # Calculate mean
  mean_val = np.mean(my_list)
  # Calculate 2.5 percentile
  percentile_025 = np.percentile(my_list, 2.5)
  # Calculate 97.5 percentile
  percentile_975 = np.percentile(my_list, 97.5)
  if percent==True:
    print(f"\n{Label}: {mean_val*100:.2f}%, (95% SI: {percentile_025*100:.2f}%, {percentile_975*100:.2f}%)")
  else:
    print(f"\n{Label}: {mean_val:.2f}, (95% SI: {percentile_025:.2f}, {percentile_975:.2f})")

In [4]:
def beta_parameters(mean, stddev):
    # make sure mean and stddev are in valid range
    assert 0 < mean < 1, "Mean should be in (0, 1) range"
    assert 0 < stddev < np.sqrt(mean * (1 - mean)), "Standard deviation should be in (0, sqrt(mean * (1 - mean))) range"
    
    # convert stddev to variance
    variance = stddev ** 2
    
    # common part of both alpha and beta
    common = mean * (1 - mean) / variance - 1
    
    # calculate alpha and beta
    alpha = mean * common
    beta = (1 - mean) * common
    
    return alpha, beta

## Load data

In [5]:
## Description of columns in "data" dataframe below

hr = pd.read_csv('ASD Mortality RR.csv')
data=pd.read_csv('Clean ASD Dataset.csv')
data.head()
# province = The name of Canadain province
# sex = sex Male/Female
# Age = Age of group increasing by 1 year increments
# population = Population size for the respective province, sex, and age group. This 2019 data was obtained from Statistics Canada.
# mortality_rate = Mortality rate per 1000 individuals from the general population. This 2019 data was obtained from Statistics Canada.
# asd_prevalence = asd prevalence from 2019 CHSCY for those 1-17 years of age.
# asd_prevalence_se = standard error for asd prevalence from 2019 CHSCY for those 1-17 years of age.
# hazard_ratio = hazard ratio for death for the respective sex. We assumed constant hazard ratio for all ages and all provinces.
# se = standard error for the hazard ratio for death in the respective sex. We assumed constant hazard ratio for all ages and all provinces.
# pop_survival = probability of survival for the respective province, sex, and age group. This variable was derived from mortality_rate column.


,province,sex,age,population,mortality_rate,asd_prevalence,asd_prevalence_se,hazard_ratio,se,pop_survival
0,Newfoundland and Labrador,Male,18,2835.0,0.4,0.039625,0.0104,2.09,0.171,0.9996
1,Newfoundland and Labrador,Male,19,2835.0,0.4,NaN,NaN,2.09,0.171,0.9996
2,Newfoundland and Labrador,Male,20,3013.0,0.4,NaN,NaN,2.09,0.171,0.9996
3,Newfoundland and Labrador,Male,21,3013.0,0.4,NaN,NaN,2.09,0.171,0.9996
4,Newfoundland and Labrador,Male,22,3013.0,0.4,NaN,NaN,2.09,0.171,0.9996


## Test

#### Conduct analysis to estimate prevalence (see README File)
This code below is manipulating a dataset based on ASD (Autism Spectrum Disorder). It's iterating through groups of data by province and sex, calculating various statistics such as ASD prevalence, mortality rate, number of ASD cases, ASD survival rate, rho_adj, and gamma_adj, and appending these values to respective lists.

In [ ]:
def run_simulation(data, hr, beta_parameters, num_simulations=1000):
    # Group the data by 'province' and 'sex' columns
    grouped = {province: data for province, data in data.groupby(['province','sex'])}

    sim_data = []

    # Initialize the tqdm progress bar
    for num_sim in tqdm(range(num_simulations), desc="Running simulations"):
        hr['current_simulated_hr'] = np.exp(np.random.normal(loc=hr['rate'], scale=hr['se']))

        # Initialize empty lists to store data
        province, sex, ages, pops, ASD_prev_3_17, N_ASD, HR, asd_survival, rho_adj, gamma_adj = ([] for _ in range(10))

        # Loop through each key in the dictionary
        for key in grouped:
            # Get ASD prevalence for the current group
            asd_prevalence_prev = grouped[key]['asd_prevalence'].values[0]
            se_ASD_prev=grouped[key]['asd_prevalence_se'].values[0]

            alpha, beta = beta_parameters(asd_prevalence_prev, se_ASD_prev)
            asd_prevalence_prev = np.random.beta(alpha, beta, 1)[0]
            #asd_prevalence_prev = np.random.normal(loc=asd_prevalence_prev, scale=se_ASD_prev)

            # Store the first ASD prevalence for the current group
            first_asd_prevalence = asd_prevalence_prev

            # Get ASD mortality rate for the current group
            mortality_rate = grouped[key]['mortality_rate'].values
            exponentiated_HR = hr[hr['Sex'] == key[1]]['current_simulated_hr'].values[0]
            
            # Get population for the current group
            population = grouped[key]['population'].values
            # Get population survival rate for the current group
            pop_survival = grouped[key]['pop_survival'].values
            # Get age for the current group
            age = grouped[key]['age'].values

            # Loop through each record in the current group
            for i, (age_val, pop_val, pop_survival_val) in enumerate(zip(age, population, pop_survival)):
                asd_mortality_rate = exponentiated_HR * pop_survival_val
                asd_survival_temp = 1 - (asd_mortality_rate / 1000)
                N_ASD_temp = pop_val * asd_prevalence_prev
                rho_adj_temp = asd_prevalence_prev * asd_survival_temp
                gamma_adj_temp = pop_val * rho_adj_temp

                # Append province, sex and age to their respective lists
                province.append(key[0])
                sex.append(key[1])
                ages.append(age_val)

                # Append population and ASD prevalence for ages 3-17 to their respective lists
                pops.append(pop_val)
                ASD_prev_3_17.append(first_asd_prevalence)
                
                # Calculate number of ASD cases and append to the list
                N_ASD.append(N_ASD_temp)
                # Calculate ASD survival rate and append to the list
                asd_survival.append(asd_survival_temp)
                # Calculate rho_adj (adjusted ASD prevalence) and append to the list
                rho_adj.append(rho_adj_temp)
                # Calculate gamma_adj (adjusted ASD cases in the population) and append to the list
                gamma_adj.append(gamma_adj_temp)
                HR.append(exponentiated_HR)
                # Update the ASD prevalence (from current ASD province, sex, age group) for the next iteration
                asd_prevalence_prev = rho_adj_temp if age_val != 89 else 0

        mydata = {'province': province, 'age': ages, 'sex': sex, 'population': pops,
                  'ASD_prev_3_17': ASD_prev_3_17, 'N_ASD': N_ASD, 'Hazard_Ratio': HR,
                  'asd_survival': asd_survival, 'rho_adj': rho_adj, 
                  'gamma_adj': gamma_adj}

        data2 = pd.DataFrame(mydata)
        sim_data.append(data2)

    return sim_data

# Use the function
if __name__ == "__main__":
    sim_data = run_simulation(data, hr, beta_parameters, num_simulations=10000)   ###   <---- In our final results, we used 10,000 simulations

Running simulations: 100%|██████████| 1000/1000 [00:06<00:00, 161.73it/s]


### Functions to aggregate simulated data by a particular group

In [7]:
# Define bins and labels
bins = [18, 20, 25, 30,35,40,45,50,55,60,65,70,75,80,85, np.inf]
labels = ['18-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54',
          '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85+']

# Add "Simulation Number" to each dataframe in the list before concatenating
for i in tqdm(range(len(sim_data)), desc="Processing simulation data"):
    sim_data[i]['Simulation Number'] = i

aggregated_sim_data = pd.concat(sim_data)

Processing simulation data:   0%|          | 0/1000 [00:00<?, ?it/s]

Processing simulation data: 100%|██████████| 1000/1000 [00:00<00:00, 11601.83it/s]


In [8]:

def get_stats_grouped(dfs, variables):
    # Define bins and labels
    bins = [18, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, np.inf]
    labels = ['18-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54',
              '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85+']

    # Add "Simulation Number" to each dataframe in the list before concatenating
    for i in tqdm(range(len(dfs)), desc="Processing simulation data"):
        dfs[i]['Simulation Number'] = i

    aggregated_sim_data = pd.concat(dfs)

    # Create a new column 'age_group' based on 'age' column
    aggregated_sim_data['age_group'] = pd.cut(aggregated_sim_data['age'], bins=bins, labels=labels, right=False)
    aggregated_sim_data = aggregated_sim_data.reset_index(drop=True)
    aggregated_sim_data = aggregated_sim_data[variables+['population','gamma_adj']+["Simulation Number"]].groupby(variables+["Simulation Number"]).sum().reset_index()
    aggregated_sim_data['rho_adj'] = aggregated_sim_data['gamma_adj']/aggregated_sim_data['population']

    # Apply the aggregation functions to each group
    agg_funcs = ['mean', 'min', 'max', lambda x: np.percentile(x, 2.5), lambda x: np.percentile(x, 97.5)]
    agg_func_names = ['Mean', 'Minimum', 'Maximum', '2.5th_quintile', '97.5th_quintile']

    descriptive_stats = aggregated_sim_data[variables+['rho_adj']].groupby(variables).agg(agg_funcs).reset_index()

    # Now the columns will be MultiIndex, so you'll need to flatten them and rename
    descriptive_stats.columns = descriptive_stats.columns.map('{0[0]}_{0[1]}'.format)

    for i, func_name in enumerate(agg_func_names):
        descriptive_stats.rename(columns={
            f'rho_adj_{func_name}': f'rho_adj_{i}'
        }, inplace=True)

    return descriptive_stats


#### Nationally

In [9]:
def get_stats(df, column):
    """
    Given a dataframe and column name, this function will return
    the mean, 2.5th percentile, 97.5th percentile, min, and max of the column.
    These statistics will be returned as percentages.
    """
    mean = df[column].mean() * 100
    percentile_2_5 = np.percentile(df[column], 2.5) * 100
    percentile_97_5 = np.percentile(df[column], 97.5) * 100
    min_val = df[column].min() * 100
    max_val = df[column].max() * 100

    return {
        "mean": f'{mean:.2f}%',
        "2.5th SI": f'{percentile_2_5:.2f}%',
        "97.5th SI": f'{percentile_97_5:.2f}%',
        "min": f'{min_val:.2f}%',
        "max": f'{max_val:.2f}%',
    }

In [15]:
def to_percent(my_data):
    # Identify numeric columns
    numeric_columns = my_data.select_dtypes(include=['float', 'int']).columns

    # Function to format string values as percent
    format_percent = lambda x: '{:.2%}'.format(float(x))

    # Apply percent formatting only to numeric columns
    my_data[numeric_columns] = my_data[numeric_columns].map(format_percent)
    return my_data

## Results

#### 1. National results 

In [12]:
variables=["Simulation Number"]

# Add "Simulation Number" to each dataframe in the list before concatenating
for i in tqdm(range(len(sim_data)), desc="Processing simulation data"):
    sim_data[i]['Simulation Number'] = i

aggregated_sim_data = pd.concat(sim_data)
aggregated_sim_data=aggregated_sim_data.reset_index(drop=True)
aggregated_sim_data = aggregated_sim_data[variables+['population','gamma_adj']].groupby(variables).sum().reset_index()
aggregated_sim_data['rho_adj']=aggregated_sim_data['gamma_adj']/aggregated_sim_data['population']

rho_adj_stats = get_stats(aggregated_sim_data, 'rho_adj')
pd.DataFrame([rho_adj_stats])

Processing simulation data: 100%|██████████| 1000/1000 [00:00<00:00, 56486.66it/s]


,mean,2.5th SI,97.5th SI,min,max
0,1.81%,1.64%,1.98%,1.56%,2.08%


#### 2. Natioanl results by sex

In [16]:
variables=['sex']
my_data = get_stats_grouped(sim_data, variables=variables)
to_percent(my_data)

Processing simulation data: 100%|██████████| 1000/1000 [00:00<00:00, 61072.02it/s]


,sex_,rho_adj_mean,rho_adj_min,rho_adj_max,rho_adj_<lambda_0>,rho_adj_<lambda_1>
0,Female,0.73%,0.49%,0.98%,0.58%,0.91%
1,Male,2.91%,2.38%,3.48%,2.60%,3.21%


#### 3. Provincial results

In [17]:
variables=['province']
my_data = get_stats_grouped(sim_data, variables=variables)
to_percent(my_data)

Processing simulation data: 100%|██████████| 1000/1000 [00:00<00:00, 66609.03it/s]


,province_,rho_adj_mean,rho_adj_min,rho_adj_max,rho_adj_<lambda_0>,rho_adj_<lambda_1>
0,Alberta,1.85%,1.07%,2.82%,1.34%,2.43%
1,British Columbia,1.98%,1.15%,2.92%,1.46%,2.58%
2,Manitoba,1.56%,0.62%,3.19%,0.95%,2.40%
3,New Brunswick,3.65%,1.97%,5.62%,2.47%,5.06%
4,Newfoundland and Labrador,1.97%,0.84%,3.78%,1.14%,3.05%
5,Northwest Territories,1.88%,0.37%,5.53%,0.70%,3.72%
6,Nova Scotia,1.34%,0.45%,2.74%,0.74%,2.08%
7,Nunavut,1.87%,0.36%,5.06%,0.69%,3.75%
8,Ontario,1.91%,1.57%,2.31%,1.70%,2.15%
9,Prince Edward Island,3.24%,1.72%,5.80%,2.28%,4.34%


#### 4. Provincial results by sex

In [18]:
variables=['province', 'sex']
my_data = get_stats_grouped(sim_data, variables=variables)
to_percent(my_data)

Processing simulation data: 100%|██████████| 1000/1000 [00:00<00:00, 56811.83it/s]


,province_,sex_,rho_adj_mean,rho_adj_min,rho_adj_max,rho_adj_<lambda_0>,rho_adj_<lambda_1>
0,Alberta,Female,0.82%,0.19%,1.85%,0.39%,1.41%
1,Alberta,Male,2.87%,1.59%,4.60%,2.02%,3.89%
2,British Columbia,Female,0.71%,0.14%,1.78%,0.35%,1.16%
3,British Columbia,Male,3.29%,1.94%,5.08%,2.32%,4.43%
4,Manitoba,Female,0.38%,0.04%,1.54%,0.08%,0.83%
5,Manitoba,Male,2.74%,1.00%,5.99%,1.61%,4.37%
6,New Brunswick,Female,0.93%,0.09%,3.04%,0.27%,1.99%
7,New Brunswick,Male,6.43%,3.43%,10.75%,4.19%,9.12%
8,Newfoundland and Labrador,Female,0.31%,0.01%,1.11%,0.08%,0.70%
9,Newfoundland and Labrador,Male,3.69%,1.47%,7.50%,2.03%,5.80%
